### Code reference :-
https://towardsdatascience.com/how-to-build-a-real-time-twitter-analysis-using-big-data-tools-dd2240946e64

In [2]:
#Twitter feed related imports

# For sending GET requests from the API
import requests

# For saving access tokens and for file management when creating and adding to the dataset
import os

# For dealing with json responses we receive from the API
import json

# For displaying the data after
import pandas as pd

# For saving the response data in CSV format
import csv

# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata

In [3]:
def auth():
    os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAG7DfwEAAAAAbJLqeNiJMbEMtiCNAc6X3xqUPXE%3DcGSmQEHDbvKKSfbcVNbYrbphIGvDkeZi4wNltyU7mmbGRFvVZq'
    return os.environ['TOKEN']


In [4]:
def create_headers(token):
    headers = {"Authorization":"Bearer {}".format(bearer_token)}
    return headers

In [5]:
def create_url(key,start_date,end_date,max_results=10):
    search_url = "https://api.twitter.com/2/tweets/search/recent"
    query_params={'query':key,
                  'start_time':start_date,
                  'end_time':end_date,
                  'max_results':max_results,
                  'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                  'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                  'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                  'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                  'next_token': {}}
    return (search_url, query_params)

In [6]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [7]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "fire lang:en"
start_time = "2022-08-07T05:26:00.000Z"
end_time = "2022-08-14T04:26:00.000Z"
max_results = 15

In [19]:
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])
json_str=json.dumps(json_response, indent=4, sort_keys=True)
print(json_str)

Endpoint Response Code: 200
{
    "data": [
        {
            "author_id": "1054523821955432448",
            "conversation_id": "1558671250779766784",
            "created_at": "2022-08-14T04:25:59.000Z",
            "id": "1558671250779766784",
            "lang": "en",
            "public_metrics": {
                "like_count": 1,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "reply_settings": "everyone",
            "source": "Twitter for Android",
            "text": "GOP under fire for rhetoric over IRS\n\nhttps://t.co/6DtyS9fySI"
        },
        {
            "author_id": "1464744692139143168",
            "conversation_id": "1558671247726088192",
            "created_at": "2022-08-14T04:25:58.000Z",
            "id": "1558671247726088192",
            "lang": "en",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_c

In [21]:
data = json.loads(json_str)
df = pd.DataFrame.from_dict(data,orient='index')
df.to_csv('tweet_feed.csv')